# **ALGORITMO VORAZ CON ACTUALIZACIONES DE RANGO UNO**

En este cuaderno se exponen los ejemplos presentados en la memoria *Análisis de un algoritmo voraz con actualizaciones de rango uno para la resolución de sistemas lineales de altas dimensiones*

Se estructura de la siguiente forma: 


1.   Primero se presenta un ejemplo donde se utiliza el algoritmo GROU para aproximar la representación separada de un vector alreatorio dado.

2.   Luego se resuelve un sistema de ecuaciones mediante el algoritmo GROU

3.   Por último se utiliza el algoritmo para aproximar la solución de la ecuación de Poisson con condiciones Dirichlet. En este último se aprovecha la estructura Laplaciana de la matriz de coeficientes.


Se presentan las fucniones que vamos a utlizar:

Como nos hemos limitado a trabajar sobre matrices y vectores, se ha diseñado una función que realice el producto de Kronecker entre estos objetos.

In [ ]:
def kronecker_product(A, B):
    # nos aseguramos de que los objetos tengan dos dimensiones, aunque una de ellas sea 1
    A = np.atleast_2d(A)
    B = np.atleast_2d(B)

    # se inicializa el resultado
    C = np.zeros((np.shape(A)[0] * np.shape(B)[0], np.shape(A)[1] * np.shape(B)[1]))
    # se itera sobre las filas de A
    for i in range(np.shape(A)[0]):
        # se itera sobre las columnas de A
        for j in range(np.shape(A)[1]):
            # se itera sobre las filas de B
            for k in range(np.shape(B)[0]):
                # se itera sobre las columnas de B
                for l in range(np.shape(B)[1]):
                    C[i * np.shape(B)[0] + k, j * np.shape(B)[1] + l] = A[i, j] * B[k, l]
    return C

def kronecker_product_l(list):
    # devuelve el producto de Kronecker de una lista de matrices
    if len(list) == 1:
        return list[0]
    else:
        return kronecker_product(list[0], kronecker_product_l(list[1:]))  # de forma recursiva se llama a sí misma


Necesitamos definir la función de error que se utiliza.

In [ ]:
def error_als3(x, a, d):
    # esta función calcula el error del problema
    # como el productorio de la diferencia entre cada
    # componente de x y a
    error = 1
    for i in range(d):
        error = error * np.linalg.norm(x[i] - a[i])
        # print(error)
    return error

def error_als(x,a,d):
    return np.linalg.norm(x-a)



Se define el algoritmo **ALS** (*Alternating Least Squares*) que vamos a usar para resolver el problema de minimización en cada iteración del algoritmo **GROU**

\begin{equation}
\min _{\left(\mathbf{x}_1, \ldots, \mathbf{x}_d\right) \in \mathbb{R}^{N_1} \times \cdots \times \mathbb{R}^{N_d}}\left\|\mathbf{r}_n-A\left(\mathbf{x}_1 \otimes \cdots \otimes \mathbf{x}_d\right)\right\|_2
\end{equation}

In [ ]:
def ALS3(f,A,itera_max,tol,N):
    try:
        d = len(N)
        # esta función devuelve el vector x que minimiza el problema de mínimos cuadrados
        # | Ax - f |
        # mediante el algoritmo ALS3
        # inicializamos el vector x

        x = [np.random.rand(N[i], 1) for i in range(d)]
        # inicializamos el contador de iteraciones
        itera = 0
        # iteramos hasta alcanzar el número máximo de iteraciones o hasta que el error sea menor que la tolerancia
        while itera < itera_max:
            # movemos el actual vector x al vector a
            a = x.copy()
            if itera == 0:
                x = a.copy()
            # iteramos sobre cada componente del vector x
            for k in range(d):
                # calculamos la matriz Z que es la matriz con
                # el producto de kronecker de las componentes
                # del vector x (ya actualizadas) excepto la k-ésima. A partir
                # de la k-ésima componente, usamos las componentes
                # del vector a (son las componentes que aún no hemos actualizado)
                Z = np.eye(N[k])
                for i in range(d):
                    if i < k:
                        # se premultiplica por x[i]
                        Z = kronecker_product(x[i], Z)
                    elif i > k:
                        # se posmultiplica por a[i]
                        Z = kronecker_product(Z, a[i])

                    else:
                        pass
                # sale del producto de kronecker correspondiente a la componente k
                # luego la premultiplicamos por la matriz A

                Z = matriz_x_matrix(A, Z)    
                # producto con A correspondiente a la componente k
                # calculamos la pseudo inversa de la matriz Z
                # Z_pinv = pseudoinv2(Z)
                Z_pinv = pseudoinv(Z)
                # actualizamos el nuevo vector x_k
                x[k] = matriz_x_matrix(Z_pinv, f)
                x[k] = x[k].reshape(N[k], 1)

            # comprobamos si el error es menor que la tolerancia
            if error_als3(x, a, d) < tol:
                print(itera, 'iteraciones realizadas')
                return x
            itera += 1
        print(itera, 'iteraciones realizadas')
        return x
    except KeyboardInterrupt:
        print(itera, 'iteraciones realizadas')
        return x

Se define el algoritmo **GROU**, el *algoritmo voraz con actualizaciones de rango uno* que usaremos para resolver los ejemplos

In [ ]:
def GROU(f, A, e, tol, rank_max, itera_max, N, inner_procedure):
    """
    Esta función resuelve el sistema lineal Ax = f usando el algoritmo GROU con representación separada del vector incógnita.

    Parámetros
    ----------
    f : numpy.ndarray
        La parte derecha de la igualdad, vector de términos independientes del sistema.
    A : numpy.ndarray
        La matriz de coeficientes del sistema lineal.
    e : float
        La tolerancia sobre la norma del residuo.
    tol : float
        La tolerancia sobre norma de la variación del residuo.
    rank_max : int
        Máximo rango de separación del tensor x.
    itera_max : int
        Máximo número de iteraciones del algoritmo ALS.

    Salida
    -------
    tuple
        Se devuelve una tupla (u, r_norm) 
        donde u es la representación separada del vector desconocido 
        y r_norm es la norma del residuo. 

    Notas
    -------

    > El algoritmo va a tratar de encontrar una actualización satisfactoria 
    para u, en el caso en que no se encuentre porque el procedimiento 
    de minimización no reduzca la norma entonces se intenta de nuevo con un 
    nuevo punto inicial (según la teoría esto puede ocurrir porque no se han realizado
    las suficientes iteraciones, o bien porque el mínimo local que se ha encontrado no 
    es suficiente para reducir la norma del residuo) 

    > Si después de 10 iteraciones no se reduce la norma del residuo, se supone
    que las iteraciones internas (por ejemplo, de ALS3) necesarias para reducirla 
    no son rentables y nos conformamos con el residuo actual. 

    """
    try:
        # si el procedimiento es ALS4 será porque le estamos pasando la matriz en 
        # representación separada, entonces se conforma la matriz para productos con 'y'
        if inner_procedure == ALS4:
            print('procedimiento ALS4')
            A_exp = conforma_matriz(A, N)
        elif inner_procedure == ALS3:
            print('procedimiento ALS3')
            A_exp = A
        else:
            raise TypeError('el procedimiento debe ser ALS3 o ALS4')
        # Inicializamos el residuo, el primer elemento r[0] va a ser el vector de 
        # términos independientes, el segundo va a ser el residuo en cada iteración
        r = [0, f.copy()]
        u = np.zeros(np.shape(f)[0])
        # aseguramos que u es un vector columna
        u = u.reshape(np.shape(u)[0], 1)
        OK = 0
        MAL = 0
        ESTABLE = 0
        # Hasta que se alcance el rango máximo....
        for i in range(rank_max):
            # Se actualiza el residuo, cambiamos el nuevo por el viejo para actualizar siempre el penúltimo residuo.
            r[0], r[1] = r[1], r[0]
            # ejecutamos el procedimiento minimizante para encontrar la actualización de la solución
            y = kronecker_product_l(inner_procedure(r[0], A, itera_max, tol, N))
            # actualizamos el residuo sobreescribiendo el que ya no se necesitará
            r[1] = r[0] - array_vector_product(A_exp, y)

            # se imprime la iteración externa (GROU) y la norma actual del residuo
            print('(', i+1, ') ', 'norma del residuo: ', ln.norm(r[1]))
            if ln.norm(r[1]) < ln.norm(r[0]):
                OK += 1
                # se imprime 
                print('iteración satisfactoria')
                u = u + y
                print('u', u.shape)
            else:
                print('iteración ha ido mal, no se reduce residuo')
                print('se intenta de nuevo...')
                # si ha ido mal la actualización por ALS* entonces se intenta otra vez
                # por si no se hubiera llegado a un mínimo satisfactorio
                # volvemos a usar el mismo residuo y no actualizamos la solución
                r[0], r[1] = r[1], r[0]
                MAL += 1
                if MAL > 10:
                    # pero si no hay mejora en unas cuantas iteraciones, entonces se 
                    # supone que ya no va a haberla en la práctica
                    return u, ln.norm(r[1])
            # si todo ha ido bien y el algoritmo converge o bien no hay suficiente mejora, entoces se termina
            # el proceso y devolvemos la solución y la norma del residuo
            if ln.norm(r[1]) < e or abs(ln.norm(r[1]) - ln.norm(r[0])) < tol:
                print(OK, MAL)
                return u, ln.norm(r[1])

        # si el algoritmo no converge se devuelve también la solución y la norma del residuo
        print(OK, MAL)
        return u, ln.norm(r[1])
    # opción 'parada de emergencia'
    except KeyboardInterrupt:
        print('Detectado KeyboardInterrupt.')
        print('Número de mejoras en residuo: ', OK)
        print('Número de empeoramientos en residuo: ', MAL)
        return 'Interrupted'

Definimos el mismo algoritmo con una modificación en caso de que la matriz venga en representación separada. Podemos utilizar las propiedades del producto de Kronecker para simplificar los cálculos.

In [ ]:
def sum_matrices(list):
    # returns sum of matrices in list
    if len(list) == 1:
        return list[0]
    else:
        return list[0] + sum_matrices(list[1:])


def separated_product(A, B, N):
    # this function returns the product of two matrices A and B
    # that are given in separated representation with the dimensions N
    C = []
    for A_sub in A:
        for B_sub in B:
            C.append(kronecker_product_l([np.dot(A_sub[i], B_sub[i]) for i in range(len(A_sub))]))
    return sum_matrices(C)


In [ ]:
def GROU_sep(f, A, e, tol, rank_max, itera_max, N, inner_procedure):
    """
    Esta función resuelve el sistema lineal Ax = f usando el algoritmo GROU con representación separada del vector incógnita.

    Parámetros
    ----------
    f : numpy.ndarray
        La parte derecha de la igualdad, vector de términos independientes del sistema.
    A : numpy.ndarray
        La matriz de coeficientes del sistema lineal.
    e : float
        La tolerancia sobre la norma del residuo.
    tol : float
        La tolerancia sobre norma de la variación del residuo.
    rank_max : int
        Máximo rango de separación del tensor x.
    itera_max : int
        Máximo número de iteraciones del algoritmo ALS.

    Salida
    -------
    tuple
        Se devuelve una tupla (u, r_norm)
        donde u es la representación separada del vector desconocido
        y r_norm es la norma del residuo.

    Notas
    -------

    > El algoritmo va a tratar de encontrar una actualización satisfactoria
    para u, en el caso en que no se encuentre porque el procedimiento
    de minimización no reduzca la norma entonces se intenta de nuevo con un
    nuevo punto inicial (según la teoría esto puede ocurrir porque no se han realizado
    las suficientes iteraciones, o bien porque el mínimo local que se ha encontrado no
    es suficiente para reducir la norma del residuo)

    > Si después de 10 iteraciones no se reduce la norma del residuo, se supone
    que las iteraciones internas (por ejemplo, de ALS3) necesarias para reducirla
    no son rentables y nos conformamos con el residuo actual.

    """
    try:
        # si el procedimiento es ALS4 será porque le estamos pasando la matriz en
        # representación separada, entonces se conforma la matriz para productos con 'y'
        if inner_procedure == ALS4:
            print('procedimiento ALS4')
        elif inner_procedure == ALS3:
            print('procedimiento ALS3')
        else:
            raise TypeError('el procedimiento debe ser ALS3 o ALS4')
        # aquí inner_procedure es siempre ALS4 pero mantengo el TypeError
        inner_procedure = ALS4
        # Inicializamos el residuo, el primer elemento r[0] va a ser el vector de
        # términos independientes, el segundo va a ser el residuo en cada iteración
        r = [0, f.copy()]
        u = np.zeros(np.shape(f)[0])
        # aseguramos que u es un vector columna
        u = u.reshape(np.shape(u)[0], 1)
        OK = 0
        MAL = 0
        ESTABLE = 0
        # Hasta que se alcance el rango máximo....
        for i in range(rank_max):
            # Se actualiza el residuo, cambiamos el nuevo por el viejo para actualizar siempre el penúltimo residuo.
            r[0], r[1] = r[1], r[0]
            # ejecutamos el procedimiento minimizante para encontrar la actualización de la solución
            y = [inner_procedure(r[0], A, itera_max, tol, N)]
            # actualizamos el residuo sobreescribiendo el que ya no se necesitará
            for ele in range(len(y)):
                y[0][ele] = y[0][ele].reshape(np.shape(y[0][ele])[0], 1)
            aux_y = np.array(separated_product(A, y, N))
            r[1] = r[0] - aux_y.reshape(aux_y.shape[0],)

            # se imprime la iteración externa (GROU) y la norma actual del residuo

            if ln.norm(r[1]) < ln.norm(r[0]):
                OK += 1
                # se imprime
                print('iteración satisfactoria')
                # se imprime la iteración externa (GROU) y la norma actual del residuo
                print('(', i + 1, ') ', 'norma del residuo: ', ln.norm(r[1]))
                u = u + kronecker_product_l(y[0])
                print('u', u.shape)
                # se reinitializa el contador de iteraciones MAL
                MAL = 0
            else:
                print('iteración ha ido mal, no se reduce residuo')
                print('se intenta de nuevo...')
                # se imprime la iteración externa (GROU) y la norma actual del residuo
                print('norma del residuo actual ', ln.norm(r[1]))
                # si ha ido mal la actualización por ALS* entonces se intenta otra vez
                # por si no se hubiera llegado a un mínimo satisfactorio
                # volvemos a usar el mismo residuo y no actualizamos la solución
                r[0], r[1] = r[1], r[0]
                MAL += 1
                if MAL > 5:
                    # pero si no hay mejora en unas cuantas iteraciones, entonces se
                    # supone que ya no va a haberla en la práctica
                    return u, ln.norm(r[1])

            # el proceso y devolvemos la solución y la norma del residuo
            if ln.norm(r[1]) < e or abs(ln.norm(r[1]) - ln.norm(r[0])) < tol:
                print(OK, MAL)
                return u, ln.norm(r[1])

        # si el algoritmo no converge se devuelve también la solución y la norma del residuo
        print(OK, MAL)
        return u, ln.norm(r[1])
    # opción 'parada de emergencia'
    except KeyboardInterrupt:
        print('Detectado KeyboardInterrupt.')
        print('Número de mejoras en residuo: ', OK)
        print('Número de empeoramientos en residuo: ', MAL)
        return 'Interrupted'


Vamos con los ejemplos.

En primer lugar se utiliza el algoritmo GROU sin representación separada